# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

#def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
#    if lines is None:
#        return
#    right_lines=[]
#    left_lines=[]
#    for line in lines:
#        for x1,y1,x2,y2 in line:
    #        cv2.line(img,(x1,y1),(x2,y2),color,thickness)
#            if ((y2-y1)/(x2-x1)) > 0:
#                right_lines.append(line)
#            else:
#                left_lines.append(line)
#    for line in right_lines:
#        for x1,y1,x2,y2 in line:
#            print("right: ")
#            print(line)
#            cv2.line(img,(x1,y1),(x2,y2),color,thickness)
#    for line in left_lines:
#        for x1,y1,x2,y2 in line:
#            print("left: ")
#            print(line)
#            cv2.line(img,(x1,y1),(x2,y2),[0,255,0],thickness)
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if lines is None:
        return
    right_lines=[]
    left_lines=[]
    leftSlopeTotal=0
    rightSlopeTotal=0
    for line in lines:
        for x1,y1,x2,y2 in line:
            #cv2.line(img,(x1,y1),(x2,y2),[0,0,255],thickness)
            #cv2.circle(img,(x1,y1), 20, (0,0,255), -1)
            slope=((y2-y1)/(x2-x1))
            if slope > 0.5:
                right_lines.append([x1,y1,x2,y2])
                rightSlopeTotal+=slope
            elif slope < -0.5:
                left_lines.append([x1,y1,x2,y2])
                leftSlopeTotal+=slope
                
    #print("Left: "+str(len(left_lines)))
    #print("Right: "+str(len(right_lines)))
    
    #
    
    #print(leftSlopeAverage)
    #print(rightSlopeAverage)
    #x1,y1,x2,y2
    minRightX=0
    maxRightX=0
    minRightY=0
    maxRightY=0
    minLeftX=0
    maxLeftX=0
    minLeftY=0
    maxLeftY=0
    if len(right_lines)>0:
    #if True:
        minRightX=min(rline[0] for rline in right_lines)
        maxRightX=max(rline[2] for rline in right_lines)
        minRightY=min(rline[1] for rline in right_lines)
        maxRightY=max(rline[3] for rline in right_lines)
        
    
    if len(left_lines)>0:
    #if True:
        #Left line
        minLeftX=min(lline[0] for lline in left_lines)
        maxLeftX=max(lline[2] for lline in left_lines)
        minLeftY=min(lline[3] for lline in left_lines)
        maxLeftY=max(lline[1] for lline in left_lines)
        
        leftSlopeAverage=((minLeftY-maxLeftY)/(maxLeftX-minLeftX))
        #leftSlopeAverage=leftSlopeTotal/len(left_lines)
        #
        leftB=maxLeftY-leftSlopeAverage*minLeftX
        cv2.line(img,(int((max([maxRightY,maxLeftY])-leftB)/leftSlopeAverage),max([maxRightY,maxLeftY])),(int((min([minRightY,minLeftY])-leftB)/leftSlopeAverage) ,min([minRightY,minLeftY])),color,thickness)
        #cv2.circle(img,(minLeftX,max([maxRightY,maxLeftY])), 10, (255,0,0), -1)
    
    if len(right_lines)>0:
        
        #rightSlopeAverage=rightSlopeTotal/len(right_lines)
        rightSlopeAverage=((maxRightY-minRightY)/(maxRightX-minRightX))
        #leftSlopeAverage=leftSlopeTotal/len(left_lines)
        #
        rightB=maxRightY-rightSlopeAverage*maxRightX
        cv2.line(img,(int((max([maxRightY,maxLeftY])-rightB)/rightSlopeAverage),max([maxRightY,maxLeftY])),(int((min([minRightY,minLeftY])-rightB)/rightSlopeAverage),min([minRightY,minLeftY])),color,thickness)
        #cv2.circle(img,(maxRightX,max([maxRightY,maxLeftY])), 10, (255,0,0), -1)
    #if len(left_lines) < len(right_lines):(int((max([maxRightY,maxLeftY])-rightB)/rightSlopeAverage)
        #print("left is broken")
    #else:
        #print("right is broken")
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
for filename in os.listdir("test_images/"):
    if os.path.isfile("test_images/"+filename):
        image = mpimg.imread('test_images/'+filename)
        ysize = image.shape[0]
        xsize = image.shape[1]
        # Note: always make a copy rather than simply using "="
        color_select = np.copy(image)
        grayImage=grayscale(color_select)
        #grayImage=np.copy(image)
        red_threshold = 200
        green_threshold = 200
        blue_threshold = 70
        rgb_threshold = [red_threshold, green_threshold, blue_threshold]
        # Identify pixels below the threshold
        thresholds = (image[:,:,0] < rgb_threshold[0]) \
                    | (image[:,:,1] < rgb_threshold[1]) \
                    | (image[:,:,2] < rgb_threshold[2])
        grayImage[thresholds] = [0]
    #    plt.imshow(grayImage,cmap='Greys_r')
    #    plt.show()
        
        mask = np.zeros_like(grayImage)   
        ignore_mask_color = 255
        imshape = image.shape
        vertices = np.array([[(0,imshape[0]),(480, 340), (580, 340), (imshape[1],imshape[0])]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_gray = cv2.bitwise_and(grayImage, mask)
        #plt.imshow(masked_gray,cmap='Greys_r')
        #plt.show()
        blurred=gaussian_blur(masked_gray,5)
        edges=canny(blurred,100,200)
        #plt.imshow(edges,cmap='Greys_r')
        #plt.show()
        #plt.imshow(masked_edges,cmap='Greys_r')
        #plt.show()
        # Define the Hough transform parameters
        # Make a blank the same size as our image to draw on
        rho = 2 # distance resolution in pixels of the Hough grid
        theta = np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 15    # minimum number of votes (intersections in Hough grid cell)
        min_line_length = 40 #minimum number of pixels making up a line
        max_line_gap = 20    # maximum gap in pixels between connectable line segments
        line_image = np.copy(image)*0 # creating a blank to draw lines on
        
        
        # Run Hough on edge detected image
        # Output "lines" is an array containing endpoints of detected line segments
        line_image = hough_lines(edges, rho, theta, threshold,
                                    min_line_length, max_line_gap)
        # Iterate over the output "lines" and draw lines on a blank image
        
        
        
        # Draw the lines on the edge image
        lines_edges = weighted_img(line_image,image, 0.8,  1, 0) 
        plt.imshow(lines_edges)
        #plt.imshow(edges,cmap='Greys_r')
        plt.show()
        mpimg.imsave("test_images/results/"+filename.replace('jpg','png'), lines_edges)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    ysize = image.shape[0]
    xsize = image.shape[1]
    # Note: always make a copy rather than simply using "="
    color_select = np.copy(image)
    grayImage=grayscale(color_select)
    #grayImage=np.copy(image)
    red_threshold = 200
    green_threshold = 200
    blue_threshold = 70
    rgb_threshold = [red_threshold, green_threshold, blue_threshold]
    # Identify pixels below the threshold
    thresholds = (image[:,:,0] < rgb_threshold[0]) \
                | (image[:,:,1] < rgb_threshold[1]) \
                | (image[:,:,2] < rgb_threshold[2])
    grayImage[thresholds] = [0]
    #plt.imshow(grayImage,cmap='Greys_r')
    #plt.show()
    
    mask = np.zeros_like(grayImage)   
    ignore_mask_color = 255
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(480, 320), (580, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_gray = cv2.bitwise_and(grayImage, mask)
    #plt.imshow(masked_gray,cmap='Greys_r')
    #plt.show()
    blurred=gaussian_blur(masked_gray,5)
    edges=canny(blurred,100,200)
    #plt.imshow(edges,cmap='Greys_r')
    #plt.show()
    #plt.imshow(masked_edges,cmap='Greys_r')
    #plt.show()
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold =  15   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    line_image = hough_lines(edges, rho, theta, threshold,
                                min_line_length, max_line_gap)
    # Iterate over the output "lines" and draw lines on a blank image
    
    
    
    # Draw the lines on the edge image
    lines_edges = weighted_img(line_image,image, 0.8,  1, 0) 
    return lines_edges

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))